In [2]:
import h5py
import pandas as pd
import numpy as np

# HDF

### 简述

h5文件格式，即 HDF 的版本 5，HDF 版本 5 不与 HDF 版本 4 及早期版本兼容。HDF, Hierarchical Data Format，可以存储不同类型的图像和数码数据的文件格式，并且可以在不同类型的机器上传输，同时还有统一处理这种文件格式的函数库。大多数普通计算机都支持这种文件格式。HDF是美国国家高级计算应用中心(National Center for Supercomputing Application, NCSA)为了满足各种领域研究需求而研制的一种能高效存储和分发科学数据的新型数据格式。HDF可以表示出科学数据存储和分布的许多必要条件。

#### HDF的优点有：

- 自述性：对于一个HDF文件里的每一个数据对象，有关于该数据的综合信息（元数据）；在没有任何外部信息的情况下，HDF允许应用程序解释HDF文件的结构和内容

- 通用性：许多数据类型都可以被嵌入在一个HDF文件里；例如，通过使用合适的HDF数据结构，可以使符号、数字和图形数据同时存储在一个HDF文件里

- 灵活性：HDF允许用户把相关的数据对象组合在一起，放到一个分层结构中，向数据对象添加描述和标签；它还允许用户把科学数据放到多个HDF文件里

- 扩展性：HDF极易容纳将来新增加的数据模式，容易与其他标准格式兼容

- 跨平台性：HDF是一个与平台无关的文件格式；HDF文件无需任何转换就可以在不同平台上使用。


### HDF 的文件格式

HDF格式可以被看成一本带目录的多章节书。HDF文件作为“数据书”，每一章包含不同类型的数据元素，HDF文件用“数据目录”列出数据元素。

一个 HDF 文件应包括一个文件头，一个或多个描述符块，若干个数据元素。文件头是用来确定一个文件是否为HDF文件，描述符块存有数据描述符的序号；一个数据描述符和数据元素一起组成了数据对象；数据对象是 HDF 的基本结构

**文件头**

文件头占用文件的前4个字节，它由4个ASCII码形式的控制字符组成，每个字符占用一个字节。第一个控制字符是‘N’，第二个是‘C’， 第三个是‘S’，第四为‘A’，即(^N^C^S^A).

**数据对象**

一个数据对象是由一个数据描述符和一个数据元素组成；数据描述符包含了数据元素的类型、位置、尺度等信息，**数据元素是实际的数据资料**；HDF 这种数据组织方式可以实现 HDF 数据的自我描述

1. 数据描述符

所有的数据描述符都为 12 个字节长，它包含 4 个区域：标识符(占16-bit)，参照数(16-bit)，数据偏移量(32-bit)，数据长度(32-bit)

- 标识符：数据标识符(tag)是用来确定数据元素区数据类型的；它是16位无符号整型值 (1~65535)，如果没有与其相对应的数据元素，则 tag 为 DFTGA_NULL 或0；下面为 tag 的值说明：
    - 1~32767——HDF 结构专用
    - 32768~64999——可由用户定义
    - 65000~65535——HDF 规格扩展使用

- 参照数：HDF 文件中的每个标记符都有唯一的一个参照数与其相对应；参照数是一个16—bit无符号整型数，在数据对象中一般是不可变的。标记符和参照数相结合确定唯一的数据对象。

- 数据偏移量和长度：数据偏移量是指从文件开始位置到数据元素的起始位置所包含的字节数。长度是指整个数据元素区占用字节数。数据偏移量和长度均为 32-bit 无符号整型数

2. 数据元素

数据元素是数据对象中的裸数据部分

3. HDF 文件中的对象分组

在 HDF 文件中通常将含有相关数据的数据对象分为一组，这些数据对象组称为数据集；HDF 用户采用应用界面来处理这些数据集；例如一套8-bit的图像数据集一般有3个数据对象：一组对象用来描述数据集的成员，一组对象是图像数据，另一组对象则用来描述图像的尺度大小；一个数据对象可以同时属于多个数据集，例如包含在一个栅格图像中的调色板对象，如果它的标识号和参照值也同时包含在另一个数据集描述符中，那么则可以被另一个栅格图像调用

# h5py.File()

``` python
h5py.File(
    name,
    mode=None,
    driver=None,
    libver=None,
    userblock_size=None,
    swmr=False,
    rdcc_nslots=None,
    rdcc_nbytes=None,
    rdcc_w0=None,
    track_order=None,
    **kwds,
)
```
**Docstring**

创建一个新的 hdf5 文件对象，更多细节参见 h5py 使用者说明

- name: 磁盘上文件的名称，或类文件对象；对于用 core 驱动程序创建的文件，HDF5 还要求它是非空的

- mode:
    - r: 只读，目标文件必须已存在
    - r+: 读或写，目标文件必须已存在
    - w: 创建文件，如果文件已存在就截断
    - w-或x: 创建文件，如果文件已存在就退出
    - a: 如果文件已存在就执行读或写，不存在就创建

- driver: 要使用的驱动程序名称，合法名称有 None, 'core', 'sec2', 'stdio', 'mpio'.

- libver: 库版本范围，支持的取值有'earliest', 'v108', 'v110',  and 'latest'. 

- userblock: 
    Desired size of user block.  Only allowed when creating a new
    file (mode w, w- or x).
- swmr
    Open the file in SWMR read mode. Only used when mode = 'r'.
- rdcc_nbytes
    Total size of the raw data chunk cache in bytes. The default size
    is 1024**2 (1 MB) per dataset.
- rdcc_w0
    The chunk preemption policy for all datasets.  This must be
    between 0 and 1 inclusive and indicates the weighting according to
    which chunks which have been fully read or written are penalized
    when determining which chunks to flush from cache.  A value of 0
    means fully read or written chunks are treated no differently than
    other chunks (the preemption is strictly LRU) while a value of 1
    means fully read or written chunks are always preempted before
    other chunks.  If your application only reads or writes data once,
    this can be safely set to 1.  Otherwise, this should be set lower
    depending on how often you re-read or re-write the same data.  The
    default value is 0.75.
- rdcc_nslots
    The number of chunk slots in the raw data chunk cache for this
    file. Increasing this value reduces the number of cache collisions,
    but slightly increases the memory used. Due to the hashing
    strategy, this value should ideally be a prime number. As a rule of
    thumb, this value should be at least 10 times the number of chunks
    that can fit in rdcc_nbytes bytes. For maximum performance, this
    value should be set approximately 100 times that number of
    chunks. The default value is 521.
- track_order
    Track dataset/group/attribute creation order under root group
    if True. If None use global default h5.get_config().track_order.
- Additional keywords
    Passed on to the selected file driver.

**Type**

ABCMeta

In [42]:
f = h5py.File("./test_files/h5py_test.hdf5", "a")
filters = f.create_dataset("filters", shape=[6, 6], dtype="f")

In [43]:
x = pd.DataFrame(np.random.randint(0, 10, [6, 6]), columns=list("ABCDEF"))
filters[:] = x
f["labels"] = np.random.randint(10, 20, 6)
f.close()

In [44]:
f = h5py.File("./test_files/h5py_test.hdf5", "r")
print(f["features"][:])
print(f["labels"][:])
print(f["filters"][:])

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[16 12 10 19 10 15]
[[6. 3. 0. 2. 2. 1.]
 [0. 8. 0. 9. 3. 4.]
 [2. 3. 5. 8. 7. 0.]
 [4. 6. 2. 3. 5. 6.]
 [4. 5. 8. 7. 0. 5.]
 [2. 9. 2. 0. 9. 4.]]


#  

#  

# f.create_dataset
```python
f.create_dataset(
    name,
    shape=None,
    dtype=None,
    data=None,
    **kwds)`
```
**Docstring**
Create a new HDF5 dataset

**Args**

- name：数据集的名称（absolute or relative）传参为None时创建匿名的数据集

- shape：数据集形状，对于标量数据集使用`()`；若`data`没有传值，则`shape`需要传值

- dtype：Numpy dtype或字符串，若`data`没有传值，则`shape`需要传值，否则会重写传值的dtype，若省略则使用 `dtype('f') `

- data: 提供数据以初始化数据集

- chunks: 元组，切块形状；为True时自动，则自动切块

- maxshape：元组，使数据集可根据此形状调整大小；对于希望无限制的轴使用None


compression
    (String or int) Compression strategy.  Legal values are 'gzip',
    'szip', 'lzf'.  If an integer in range(10), this indicates gzip
    compression level. Otherwise, an integer indicates the number of a
    dynamically loaded compression filter.
    
compression_opts
    Compression settings.  This is an integer for gzip, 2-tuple for
    szip, etc. If specifying a dynamically loaded compression filter
    number, this must be a tuple of values.
    
scaleoffset
    (Integer) Enable scale/offset filter for (usually) lossy
    compression of integer or floating-point data. For integer
    data, the value of scaleoffset is the number of bits to
    retain (pass 0 to let HDF5 determine the minimum number of
    bits necessary for lossless compression). For floating point
    data, scaleoffset is the number of digits after the decimal
    place to retain; stored values thus have absolute error
    less than 0.5*10**(-scaleoffset).
    
shuffle
    (T/F) Enable shuffle filter.
fletcher32
    (T/F) Enable fletcher32 error detection. Not permitted in
    conjunction with the scale/offset filter.
fillvalue
    (Scalar) Use this value for uninitialized parts of the dataset.
track_times
    (T/F) Enable dataset creation timestamps.
track_order
    (T/F) Track attribute creation order if True. If omitted use
    global default h5.get_config().track_order.
external
    (List of tuples) Sets the external storage property, thus
    designating that the dataset will be stored in one or more
    non-HDF5 file(s) external to the HDF5 file. Adds each listed
    tuple of (file[, offset[, size]]) to the dataset's list of
    external files.

**Type**

method